# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 10

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras

#write your model here
#model a
model_a = Sequential()

model_a.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_a.add(Flatten())

model_a.add(Dense(1000, activation='relu'))
model_a.add(Dropout(0.5))

model_a.add(Dense(500, activation='relu'))
model_a.add(Dropout(0.5))

#softmax layer
model_a.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [7]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_a.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [8]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [10]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [11]:
model_a.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 5s - loss: 5.9100 - categorical_accuracy: 0.2815Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 200s 3s/step - loss: 5.8529 - categorical_accuracy: 0.2841 - val_loss: 4.7454 - val_categorical_accuracy: 0.3600

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-5.87963-0.28356-4.74543-0.36000.h5
Epoch 2/10
67/67 [==============================] - 26s 381ms/step - loss: 3.4411 - categorical_accuracy: 0.2836 - val_loss: 2.4837 - val_categorical_accuracy: 0.2800

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-3.44110-0.28358-2.48375-0.28000.h5
Epoch 3/10
67/67 [==============================] - 26s 388ms/step - loss: 1.8730 - categorical_accuracy: 0.3184 - val_loss: 1.2485 - val_categorical_accuracy: 0.5100

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-1.87303-0.31841-1.24849-0.51000.h5
Epoch 4/10
67/67 [==============================] - 27s 405ms/step - loss: 1.6609 - categorical_accuracy: 0.3433 - val_loss: 1.3044 - val_categorical_accuracy: 0.4000

Ep

In [12]:
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
channel = 3
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator_ex(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    batch_data[folder,idx] = temp #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [13]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_5 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_6 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 10, 40, 40, 64)    55360     
__________

In [14]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [15]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = Source path =  Project_data/train 10
Epoch 1/10
; batch size = 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


56/67 [========================>.....] - ETA: 20s - loss: 1.8072 - categorical_accuracy: 0.2054Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 127s 2s/step - loss: 1.7747 - categorical_accuracy: 0.2188 - val_loss: 1.6027 - val_categorical_accuracy: 0.3000

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-1.77666-0.21418-1.60269-0.30000.h5
Epoch 2/10
67/67 [==============================] - 44s 657ms/step - loss: 1.5684 - categorical_accuracy: 0.2687 - val_loss: 1.5106 - val_categorical_accuracy: 0.3000

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-1.56845-0.26866-1.51056-0.30000.h5
Epoch 3/10
67/67 [==============================] - 44s 658ms/step - loss: 1.5854 - categorical_accuracy: 0.2786 - val_loss: 1.5450 - val_categorical_accuracy: 0.3100

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-1.58536-0.27861-1.54496-0.31000.h5
Epoch 4/10
67/67 [==============================] - 44s 661ms/step - loss: 1.5304 - categorical_accuracy: 0.2935 - val_loss: 1.4809 - val_categorical_accuracy: 0.3300

Ep

##### Changing x,y,z values 


In [16]:
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

In [17]:
# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [18]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [19]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 1.6465 - categorical_accuracy: 0.1969Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 75s 1s/step - loss: 1.6454 - categorical_accuracy: 0.1956 - val_loss: 1.6093 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-1.64573-0.19759-1.60931-0.21000.h5
Epoch 2/10
67/67 [==============================] - 24s 357ms/step - loss: 1.6106 - categorical_accuracy: 0.1990 - val_loss: 1.6085 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-1.61062-0.19900-1.60854-0.23000.h5
Epoch 3/10
67/67 [==============================] - 25s 376ms/step - loss: 1.6091 - categorical_accuracy: 0.2338 - val_loss: 1.6075 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-1.60910-0.23383-1.60752-0.23000.h5
Epoch 4/10
67/67 [==============================] - 24s 365ms/step - loss: 1.6106 - categorical_accuracy: 0.1891 - val_loss: 1.6078 - val_categorical_accuracy: 0.2100

Epo

##### Batch size = 20 


In [20]:
train_generator = generator_ex(train_path, train_doc, 20)
val_generator = generator_ex(val_path, val_doc, 20)

In [21]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/valSource path =  Project_data/train ; batch size = 20
 ; batch size = 20
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/67 [=============>................] - ETA: 1:19 - loss: 1.5973 - categorical_accuracy: 0.1953Batch:  34 Index: 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 88s 1s/step - loss: 1.5716 - categorical_accuracy: 0.2408 - val_loss: 1.5721 - val_categorical_accuracy: 0.2200

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-1.58751-0.20866-1.57211-0.22000.h5
Epoch 2/10
67/67 [==============================] - 30s 444ms/step - loss: 1.5684 - categorical_accuracy: 0.2687 - val_loss: 1.5143 - val_categorical_accuracy: 0.4000

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-1.56836-0.26866-1.51427-0.40000.h5
Epoch 3/10
67/67 [==============================] - 31s 458ms/step - loss: 1.5136 - categorical_accuracy: 0.3433 - val_loss: 1.4296 - val_categorical_accuracy: 0.4100

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-1.51360-0.34328-1.42957-0.41000.h5
Epoch 4/10
67/67 [==============================] - 31s 457ms/step - loss: 1.4383 - categorical_accuracy: 0.3284 - val_loss: 1.3500 - val_categorical_accuracy: 0.3600

Epo

#####  Batch size  = 30 


In [22]:
train_generator = generator_ex(train_path, train_doc, 30)
val_generator = generator_ex(val_path, val_doc, 30)

In [23]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 30Source path =  Project_data/train ; batch size = 30
Epoch 1/10



/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/67 [..............................] - ETA: 5:20 - loss: 1.0995 - categorical_accuracy: 0.5667Batch:  4 Index: 30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 89s 1s/step - loss: 0.9787 - categorical_accuracy: 0.6395 - val_loss: 1.2059 - val_categorical_accuracy: 0.5375

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-0.99734-0.62138-1.20592-0.53750.h5
Epoch 2/10
67/67 [==============================] - 26s 386ms/step - loss: 1.0515 - categorical_accuracy: 0.5572 - val_loss: 1.0705 - val_categorical_accuracy: 0.5600

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-1.05150-0.55721-1.07047-0.56000.h5
Epoch 3/10
67/67 [==============================] - 24s 362ms/step - loss: 1.0676 - categorical_accuracy: 0.5622 - val_loss: 1.2628 - val_categorical_accuracy: 0.5200

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-1.06761-0.56219-1.26282-0.52000.h5
Epoch 4/10
67/67 [==============================] - 24s 359ms/step - loss: 0.9284 - categorical_accuracy: 0.6269 - val_loss: 1.0746 - val_categorical_accuracy: 0.5300

Epo

##### Batch size  = 40 


In [24]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [25]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/67 [..............................] - ETA: 6:10 - loss: 0.6568 - categorical_accuracy: 0.7625 Batch:  3 Index: 40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 189s 3s/step - loss: 0.5666 - categorical_accuracy: 0.7862 - val_loss: 1.0158 - val_categorical_accuracy: 0.5542

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-0.57187-0.78493-1.01584-0.55417.h5
Epoch 2/10
67/67 [==============================] - 127s 2s/step - loss: 0.5123 - categorical_accuracy: 0.8162 - val_loss: 1.1681 - val_categorical_accuracy: 0.5000

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-0.51278-0.81631-1.16806-0.50000.h5
Epoch 3/10
67/67 [==============================] - 124s 2s/step - loss: 0.4810 - categorical_accuracy: 0.8235 - val_loss: 0.9972 - val_categorical_accuracy: 0.6200

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-0.48101-0.82353-0.99716-0.62000.h5
Epoch 4/10
67/67 [==============================] - 123s 2s/step - loss: 0.4495 - categorical_accuracy: 0.8314 - val_loss: 1.1717 - val_categorical_accuracy: 0.5150

Epoch 00

##### using Adadelta optimizer

In [26]:
model_b.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [27]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [28]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/67 [..............................] - ETA: 5:42 - loss: 0.3432 - categorical_accuracy: 0.9125Batch:  3 Index: 40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 191s 3s/step - loss: 0.3951 - categorical_accuracy: 0.8590 - val_loss: 1.5010 - val_categorical_accuracy: 0.5792

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-0.39507-0.85971-1.50098-0.57917.h5
Epoch 2/10
67/67 [==============================] - 124s 2s/step - loss: 0.2995 - categorical_accuracy: 0.8849 - val_loss: 1.5917 - val_categorical_accuracy: 0.5800

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-0.30080-0.88412-1.59168-0.58000.h5
Epoch 3/10
67/67 [==============================] - 122s 2s/step - loss: 0.2296 - categorical_accuracy: 0.9148 - val_loss: 1.5336 - val_categorical_accuracy: 0.5800

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-0.22961-0.91484-1.53358-0.58000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5.
Epoch 4/10
67/67 [==============================] - 123s 2s/step - loss: 0.1294 - categorical_accuracy: 0.9561 

##### epoch = 20 

In [29]:
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 20


In [30]:
import keras
model_b.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [31]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [32]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/67 [..............................] - ETA: 6:03 - loss: 0.0452 - categorical_accuracy: 0.9750Batch:  3 Index: 40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 185s 3s/step - loss: 0.1879 - categorical_accuracy: 0.9422 - val_loss: 2.0377 - val_categorical_accuracy: 0.5333

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-0.18199-0.94377-2.03775-0.53333.h5
Epoch 2/20
67/67 [==============================] - 125s 2s/step - loss: 0.1081 - categorical_accuracy: 0.9604 - val_loss: 2.2205 - val_categorical_accuracy: 0.5950

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-0.10971-0.95966-2.22051-0.59500.h5
Epoch 3/20
67/67 [==============================] - 124s 2s/step - loss: 0.0594 - categorical_accuracy: 0.9789 - val_loss: 2.1238 - val_categorical_accuracy: 0.6250

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-0.05942-0.97893-2.12379-0.62500.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5.
Epoch 4/20
67/67 [==============================] - 122s 2s/step - loss: 0.0440 - categorical_accuracy: 0.9868 

##### 99.39% accuracy in training data

##### model_c

In [33]:
model_c = Sequential()
model_c.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_c.add(Activation('relu'))
model_c.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Flatten())
model_c.add(Dense(512, activation='relu'))
model_c.add(BatchNormalization())
model_c.add(Dropout(0.5))
model_c.add(Dense(classes, activation='softmax'))
model_c.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_c.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_13 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_14 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [34]:
# Changing back batch size, images per frame, height and width of image
batch_size = 10
x = 30 # number of frames
y = 60 # image width
z = 60 # image height
num_epochs = 20

In [35]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [36]:
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 2.2133 - categorical_accuracy: 0.2723Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 73s 1s/step - loss: 2.1867 - categorical_accuracy: 0.2820 - val_loss: 3.0701 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-2.20577-0.27451-3.07013-0.18000.h5
Epoch 2/20
67/67 [==============================] - 23s 348ms/step - loss: 2.0307 - categorical_accuracy: 0.2736 - val_loss: 2.0495 - val_categorical_accuracy: 0.2900

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-2.03071-0.27363-2.04945-0.29000.h5
Epoch 3/20
67/67 [==============================] - 25s 373ms/step - loss: 1.9401 - categorical_accuracy: 0.3085 - val_loss: 2.1808 - val_categorical_accuracy: 0.2700

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-1.94013-0.30846-2.18082-0.27000.h5
Epoch 4/20
67/67 [==============================] - 24s 362ms/step - loss: 1.8850 - categorical_accuracy: 0.2488 - val_loss: 2.6582 - val_categorical_accuracy: 0.1900

Epo

##### model_d

In [37]:
input_shape=(x,y,z,channel)

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
# Define model
model_d = Sequential()

model_d.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_d.add(Flatten())

model_d.add(Dense(nb_dense[0], activation='relu'))
model_d.add(Dropout(0.5))

model_d.add(Dense(nb_dense[1], activation='relu'))
model_d.add(Dropout(0.5))

#softmax layer
model_d.add(Dense(nb_dense[2], activation='softmax'))
model_d.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_d.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_19 (Conv3D)           (None, 30, 60, 60, 8)     656       
_________________________________________________________________
batch_normalization_6 (Batch (None, 30, 60, 60, 8)     32        
_________________________________________________________________
activation_19 (Activation)   (None, 30, 60, 60, 8)     0         
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 15, 30, 30, 8)     0         
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 15, 30, 30, 16)    3472      
_________________________________________________________________
batch_normalization_7 (Batch (None, 15, 30, 30, 16)    64        
_________________________________________________________________
activation_20 (Activation)   (None, 15, 30, 30, 16)    0         
__________

In [38]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


66/67 [============================>.] - ETA: 1s - loss: 2.1612 - categorical_accuracy: 0.2924

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 71s 1s/step - loss: 2.1446 - categorical_accuracy: 0.2980 - val_loss: 1.3263 - val_categorical_accuracy: 0.4100

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-2.15611-0.29412-1.32629-0.41000.h5
Epoch 2/10
67/67 [==============================] - 22s 334ms/step - loss: 1.8135 - categorical_accuracy: 0.2985 - val_loss: 1.5352 - val_categorical_accuracy: 0.3300

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-1.81345-0.29851-1.53518-0.33000.h5
Epoch 3/10
67/67 [==============================] - 24s 353ms/step - loss: 1.4970 - categorical_accuracy: 0.3284 - val_loss: 1.5154 - val_categorical_accuracy: 0.4300

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-1.49699-0.32836-1.51537-0.43000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5.
Epoch 4/10
67/67 [==============================] - 23s 340ms/step - loss: 1.4273 - categorical_accuracy: 0.

##### Model with images per frame(width and height of images)

In [39]:
x = 30 # number of frames
y = 64 # image width
z = 64 # image height 

classes = 5
channel = 1

In [40]:
model_e = Sequential()
model_e.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_e.add(Activation('relu'))
model_e.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Flatten())
model_e.add(Dense(512, activation='relu'))
model_e.add(BatchNormalization())
model_e.add(Dropout(0.5))
model_e.add(Dense(classes, activation='softmax'))
model_e.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_e.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_23 (Conv3D)           (None, 30, 64, 64, 32)    896       
_________________________________________________________________
activation_23 (Activation)   (None, 30, 64, 64, 32)    0         
_________________________________________________________________
conv3d_24 (Conv3D)           (None, 30, 64, 64, 32)    27680     
_________________________________________________________________
activation_24 (Activation)   (None, 30, 64, 64, 32)    0         
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 10, 22, 22, 32)    0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 10, 22, 22, 32)    0         
_________________________________________________________________
conv3d_25 (Conv3D)           (None, 10, 22, 22, 64)    55360     
__________

In [41]:
batch_size = 5
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 20
model_e.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 5
Source path =  Project_data/train ; batch size =Epoch 1/20
 5


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 39s 582ms/step - loss: 2.1254 - categorical_accuracy: 0.2746 - val_loss: 2.2144 - val_categorical_accuracy: 0.2400

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-2.12536-0.27463-2.21442-0.24000.h5
Epoch 2/20
64/67 [===========================>..] - ETA: 1s - loss: 1.9234 - categorical_accuracy: 0.3063Batch:  133 Index: 5


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 35s 517ms/step - loss: 1.9052 - categorical_accuracy: 0.3035 - val_loss: 3.5157 - val_categorical_accuracy: 0.3200

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-1.90859-0.30514-3.51571-0.32000.h5
Epoch 3/20
67/67 [==============================] - 21s 311ms/step - loss: 1.9276 - categorical_accuracy: 0.3433 - val_loss: 4.0424 - val_categorical_accuracy: 0.1800

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-1.92757-0.34328-4.04242-0.18000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5.
Epoch 4/20
67/67 [==============================] - 23s 343ms/step - loss: 1.8104 - categorical_accuracy: 0.3184 - val_loss: 1.7374 - val_categorical_accuracy: 0.2800

Epoch 00004: saving model to model_init_2020-02-0915_11_30.104831/model-00004-1.81037-0.31841-1.73744-0.28000.h5
Epoch 5/20
67/67 [==============================] - 22s 335ms/step - loss: 1.9548 - categorical_accuracy:

##### model_f

In [42]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 1
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_f = Sequential()

model_f.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_f.add(Flatten())

model_f.add(Dense(nb_dense[0], activation='relu'))
model_f.add(Dropout(0.5))

model_f.add(Dense(nb_dense[1], activation='relu'))
model_f.add(Dropout(0.5))

#softmax layer
model_f.add(Dense(nb_dense[2], activation='softmax'))
model_f.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_f.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_29 (Conv3D)           (None, 30, 120, 120, 8)   224       
_________________________________________________________________
batch_normalization_11 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_29 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_19 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_30 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_12 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_30 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [43]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 30
model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 6.3931 - categorical_accuracy: 0.2554Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 81s 1s/step - loss: 6.3501 - categorical_accuracy: 0.2508 - val_loss: 5.0220 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-6.35119-0.25339-5.02199-0.25000.h5
Epoch 2/30
67/67 [==============================] - 24s 363ms/step - loss: 2.8264 - categorical_accuracy: 0.2388 - val_loss: 1.4508 - val_categorical_accuracy: 0.3600

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-2.82644-0.23881-1.45077-0.36000.h5
Epoch 3/30
67/67 [==============================] - 25s 369ms/step - loss: 1.7548 - categorical_accuracy: 0.3284 - val_loss: 1.2833 - val_categorical_accuracy: 0.4600

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-1.75477-0.32836-1.28334-0.46000.h5
Epoch 4/30
67/67 [==============================] - 26s 394ms/step - loss: 1.6751 - categorical_accuracy: 0.3085 - val_loss: 1.4143 - val_categorical_accuracy: 0.3500

Epo

##### model_g

In [44]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_g = Sequential()

model_g.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_g.add(Flatten())

model_g.add(Dense(nb_dense[0], activation='relu'))
model_g.add(Dropout(0.5))

model_g.add(Dense(nb_dense[1], activation='relu'))
model_g.add(Dropout(0.5))

#softmax layer
model_g.add(Dense(nb_dense[2], activation='softmax'))
model_g.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_g.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_33 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_15 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_33 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_23 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_34 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_16 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_34 (Activation)   (None, 15, 60, 60, 16)    0         
__________

##### Model_h

In [45]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_g.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size =Source path =  Project_data/train ; batch size = 10
 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 5.8213 - categorical_accuracy: 0.2785Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 86s 1s/step - loss: 5.7498 - categorical_accuracy: 0.2732 - val_loss: 2.1726 - val_categorical_accuracy: 0.3300

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-5.76806-0.27602-2.17264-0.33000.h5
Epoch 2/10
67/67 [==============================] - 26s 382ms/step - loss: 2.6280 - categorical_accuracy: 0.2537 - val_loss: 1.4409 - val_categorical_accuracy: 0.4300

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-2.62800-0.25373-1.44085-0.43000.h5
Epoch 3/10
67/67 [==============================] - 28s 414ms/step - loss: 1.8095 - categorical_accuracy: 0.3184 - val_loss: 1.7458 - val_categorical_accuracy: 0.4000

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-1.80948-0.31841-1.74579-0.40000.h5
Epoch 4/10
67/67 [==============================] - 30s 441ms/step - loss: 1.9048 - categorical_accuracy: 0.2289 - val_loss: 1.4758 - val_categorical_accuracy: 0.2500

Epo

In [46]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

# Define model
model_h = Sequential()

model_h.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_h.add(Flatten())

model_h.add(Dense(nb_dense[0], activation='relu'))
model_h.add(Dropout(0.5))

model_h.add(Dense(nb_dense[1], activation='relu'))
model_h.add(Dropout(0.5))

#softmax layer
model_h.add(Dense(nb_dense[2], activation='softmax'))
model_h.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_h.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_37 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_19 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_37 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_27 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_38 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_20 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_38 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [47]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_h.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Source path =  Project_data/train Project_data/val ; batch size = 10
Epoch 1/10
; batch size = 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 6.3324 - categorical_accuracy: 0.2631Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 87s 1s/step - loss: 6.2098 - categorical_accuracy: 0.2582 - val_loss: 4.0269 - val_categorical_accuracy: 0.2400

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-6.24593-0.26094-4.02689-0.24000.h5
Epoch 2/10
67/67 [==============================] - 25s 380ms/step - loss: 2.1908 - categorical_accuracy: 0.2786 - val_loss: 1.4041 - val_categorical_accuracy: 0.4400

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-2.19079-0.27861-1.40409-0.44000.h5
Epoch 3/10
67/67 [==============================] - 26s 393ms/step - loss: 1.7888 - categorical_accuracy: 0.3781 - val_loss: 1.4737 - val_categorical_accuracy: 0.3300

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-1.78881-0.37811-1.47374-0.33000.h5
Epoch 4/10
67/67 [==============================] - 29s 436ms/step - loss: 1.6573 - categorical_accuracy: 0.3085 - val_loss: 1.4482 - val_categorical_accuracy: 0.3200

Epo

##### model_i

In [48]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

# Define model
model_i = Sequential()

model_i.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_i.add(BatchNormalization())
model_i.add(Activation('relu'))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

model_i.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_i.add(BatchNormalization())
model_i.add(Activation('relu'))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

model_i.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_i.add(BatchNormalization())
model_i.add(Activation('relu'))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

model_i.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_i.add(Activation('relu'))
model_i.add(Dropout(0.25))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_i.add(Flatten())

model_i.add(Dense(nb_dense[0], activation='relu'))
model_i.add(Dropout(0.5))

model_i.add(Dense(nb_dense[1], activation='relu'))
model_i.add(Dropout(0.5))

#softmax layer
model_i.add(Dense(nb_dense[2], activation='softmax'))
model_i.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_41 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_23 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_41 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_31 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_42 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_24 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_42 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [49]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_i.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 1.9433 - categorical_accuracy: 0.2969Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 89s 1s/step - loss: 1.9316 - categorical_accuracy: 0.2911 - val_loss: 1.3851 - val_categorical_accuracy: 0.4000

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-1.93477-0.29412-1.38509-0.40000.h5
Epoch 2/10
67/67 [==============================] - 25s 373ms/step - loss: 1.5750 - categorical_accuracy: 0.3234 - val_loss: 1.4378 - val_categorical_accuracy: 0.3500

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-1.57500-0.32338-1.43782-0.35000.h5
Epoch 3/10
67/67 [==============================] - 28s 424ms/step - loss: 1.5308 - categorical_accuracy: 0.3781 - val_loss: 1.6499 - val_categorical_accuracy: 0.2600

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-1.53081-0.37811-1.64992-0.26000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5.
Epoch 4/10
67/67 [==============================] - 29s 426ms/step - loss: 1.3270 - categorical_accuracy: 0.

##### model_final

In [50]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(30,120,120,3)

# Define model
model_final = Sequential()

model_final.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(Activation('relu'))
model_final.add(Dropout(0.25))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_final.add(Flatten())

model_final.add(Dense(nb_dense[0], activation='relu'))
model_final.add(Dropout(0.5))

model_final.add(Dense(nb_dense[1], activation='relu'))
model_final.add(Dropout(0.5))

#softmax layer
model_final.add(Dense(nb_dense[2], activation='softmax'))

In [51]:
optimiser = keras.optimizers.Adam() #write your optimizer
model_final.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_final.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_45 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_26 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_45 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_35 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_46 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_27 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_46 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [52]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [53]:
batch_size = 10
num_epochs = 20
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 1.9103 - categorical_accuracy: 0.2354Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 87s 1s/step - loss: 1.9060 - categorical_accuracy: 0.2314 - val_loss: 1.4182 - val_categorical_accuracy: 0.4000

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-1.90824-0.23379-1.41815-0.40000.h5
Epoch 2/20
67/67 [==============================] - 26s 384ms/step - loss: 1.6067 - categorical_accuracy: 0.3085 - val_loss: 1.6995 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-1.60669-0.30846-1.69951-0.22000.h5
Epoch 3/20
67/67 [==============================] - 29s 440ms/step - loss: 1.5522 - categorical_accuracy: 0.2637 - val_loss: 1.5096 - val_categorical_accuracy: 0.3000

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-1.55220-0.26368-1.50955-0.30000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/20
67/67 [==============================] - 29s 428ms/step - loss: 1.5516 - catego

In [54]:
batch_size = 32
num_epochs = 20
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 31s 469ms/step - loss: 0.9695 - categorical_accuracy: 0.5821 - val_loss: 0.8921 - val_categorical_accuracy: 0.6300

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-0.96948-0.58209-0.89208-0.63000.h5
Epoch 2/20
67/67 [==============================] - 29s 427ms/step - loss: 0.8834 - categorical_accuracy: 0.6468 - val_loss: 0.8682 - val_categorical_accuracy: 0.6200

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-0.88342-0.64677-0.86822-0.62000.h5
Epoch 3/20
67/67 [==============================] - 27s 407ms/step - loss: 0.9828 - categorical_accuracy: 0.6368 - val_loss: 1.0422 - val_categorical_accuracy: 0.5300

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-0.98283-0.63682-1.04217-0.53000.h5
Epoch 4/20
67/67 [==============================] - 28s 422ms/step - loss: 1.0040 - categorical_accuracy: 0.6070 - val_loss: 1.0273 - val_categorical_accuracy: 0.6400



In [55]:
batch_size = 64
num_epochs = 20

train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)


model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 64Source path =  Project_data/train ; batch size = 64
Epoch 1/20



/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Batch:  2 Index: 64


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 242s 4s/step - loss: 0.5613 - categorical_accuracy: 0.8011 - val_loss: 0.7496 - val_categorical_accuracy: 0.7152

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-0.56552-0.79826-0.74963-0.71519.h5
Epoch 2/20
67/67 [==============================] - 140s 2s/step - loss: 0.5742 - categorical_accuracy: 0.7845 - val_loss: 0.6987 - val_categorical_accuracy: 0.7100

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-0.57488-0.78404-0.69867-0.71000.h5
Epoch 3/20
67/67 [==============================] - 140s 2s/step - loss: 0.5805 - categorical_accuracy: 0.7928 - val_loss: 0.8023 - val_categorical_accuracy: 0.7000

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-0.58048-0.79280-0.80232-0.70000.h5
Epoch 4/20
67/67 [==============================] - 143s 2s/step - loss: 0.5726 - categorical_accuracy: 0.7779 - val_loss: 0.6408 - val_categorical_accuracy: 0.7750

Epoch 00

In [ ]:
batch_size = 40
num_epochs = 20

train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)


model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 1/67 [..............................] - ETA: 10:21 - loss: 0.5990 - categorical_accuracy: 0.7500Batch:  3 Index: 40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 224s 3s/step - loss: 0.5652 - categorical_accuracy: 0.7936 - val_loss: 0.7251 - val_categorical_accuracy: 0.7042

Epoch 00001: saving model to model_init_2020-02-0915_11_30.104831/model-00001-0.56642-0.79362-0.72510-0.70417.h5
Epoch 2/20
67/67 [==============================] - 150s 2s/step - loss: 0.5799 - categorical_accuracy: 0.7768 - val_loss: 0.7165 - val_categorical_accuracy: 0.7300

Epoch 00002: saving model to model_init_2020-02-0915_11_30.104831/model-00002-0.57926-0.77682-0.71653-0.73000.h5
Epoch 3/20
67/67 [==============================] - 148s 2s/step - loss: 0.5562 - categorical_accuracy: 0.7928 - val_loss: 0.7271 - val_categorical_accuracy: 0.7300

Epoch 00003: saving model to model_init_2020-02-0915_11_30.104831/model-00003-0.55619-0.79280-0.72709-0.73000.h5
Epoch 4/20
67/67 [==============================] - 148s 2s/step - loss: 0.5519 - categorical_accuracy: 0.7937 - val_loss: 0.7285 - val_categorical_accuracy: 0.7250

Epoch 00

##### approx 79% accuracy